In [1]:
#the following script defines all the required functions for the g-tensor, magnetic vectors and rotations
#We run this script using jupyter Python 3.

#Load it all up
import numpy as np
from matplotlib import pyplot as plt
from numpy import linalg as LA

PI = (3.141592653589793);

#Function to create the diagonal g-tensor
def gmat(gx,gy,gz):
    return [[gx,0,0],
            [0,gy,0],
            [0,0,gz]
           ]

#function to determine the magnetic vector for given Bmag and angles
def Bvect(Bmag,theta,phi):
    return [[Bmag*np.cos(theta)*np.cos(phi)],
            [Bmag*np.sin(theta)*np.cos(phi)],
            [Bmag*np.sin(phi)]
           ]

#Define the function that rotates a 3x3 matrix about the Y-axis by angle 'th' (radians) 
def rotY(g,th):
    #g = gmat(gx,gy,gz)
    Ry = [[np.cos(th),0,np.sin(th)],
          [0,1,0],
          [-np.sin(th),0,np.cos(th)]
         ]
    RyInv = [[np.cos(th),0,-np.sin(th)],
          [0,1,0],
          [np.sin(th),0,np.cos(th)]
         ]
    a = np.dot(g,RyInv)
    b = np.dot(Ry,a)
    return b

#Define the function that rotates a 3x3 matrix about the Z-axis by angle "th" (radians) 
def rotZ(g,th):
    #g = gmat(gx,gy,gz)
    Rz = [[np.cos(th),-np.sin(th),0],
          [np.sin(th),np.cos(th),0],
          [0,0,1]
         ]
    RzInv = [[np.cos(th),np.sin(th),0],
          [-np.sin(th),np.cos(th),0],
          [0,0,1]
         ]
    a = np.dot(g,RzInv)
    b = np.dot(Rz,a)
    return b

#Define the function that rotates a 3x3 matrix about the Z-axis by angle "th" (radians)  
def rotX(g,th):
    #g = gmat(gx,gy,gz)
    Rx = [[1,0,0],
          [0,np.cos(th),-np.sin(th)],
          [0,np.sin(th),np.cos(th)]
         ]
    RxInv = [[1,0,0],
          [0,np.cos(th),np.sin(th)],
          [0,-np.sin(th),np.cos(th)]
         ]
    a = np.dot(g,RxInv)
    b = np.dot(Rx,a)
    return b



def gObserved2(X,gx,gy,gz,phi0,Theta0,vartheta0):
#This function is the same the g-tensor definition in the main text.
#The observed g-factor, for a diagonal g-tensor, rotated around the y-axis by psi.
#Calculated for an arbitrary magnetic field orientation X = [theta,phi]
#where theta is a rotation from x->y and phi is a rotation from x->z.

#we apply rotation around the y-axis by psi, and around the z axis by xi
    #theta is the in-plane angle
    #phi is the out-of-plane angle. 
    
#print('Calling gObserved Function', gx, gy, gz,psi) #PRINT line for testing
    output = []
    theta, phi = X  
    Num = len(theta)
    
    for i in range(Num):
        #create the specific B
        Bmag = 1
        B = Bvect(Bmag,theta[i],phi[i])
        
        #create the g-tensor
        g = gmat(gx,gy,gz)

        #we use Z -> Y -> Z axis rotation to follow the Euler sequence allowing fitting to any aritrary orientation.
        
        #rotate the g-tensor around z axis
        gexpA = rotZ(g,Theta0)
               
        #rotate the g-tensor around the y axis - Note: our convention is to use negative -phi0 for counterclockwise rotation (see methods)
        gexpB = rotY(gexpA,-phi0)

        #rotate the g-tensor around the z axis
        gexp = rotZ(gexpB,vartheta0)
        
        #|g| = Norm(g.B)
        #print(theta[i]) #print line for troubleshooting
        a = np.dot(gexp,B);
        output.append(LA.norm(a))
    return output

In [2]:
#the experimental data is saved in a csv file.
#this code loads the csv file with name 'filename'
#the csv file should have three collums; 
    #observed theta, phi, g-factor.

#Version for my data
import csv
#Name of the csv file to open (in path)
#filename09 = 'G409FullData.csv'
#filename07 = 'G407FullData.csv'
filename09 = 'G409FullData.csv'
filename07 = 'G407FullData.csv'

with open(filename09) as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    
    #open variables to contain the rows.
    theta_array09 = []
    phi_array09 = []
    g_array09 = []
    for row in readCSV:
        #Fill the data by reading through the file
        readTHETA = [float(row[0])]
        readPHI = [float(row[1])]
        readg = [float(row[2])]
        
        
        
        phi_array09.extend(readPHI)
        theta_array09.extend(readTHETA)
        g_array09.extend(readg)
        
with open(filename07) as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    
    #open variables to contain the rows.
    theta_array07 = []
    phi_array07 = []
    g_array07 = []
    for row in readCSV:
        #Fill the data by reading through the file
        readTHETA = [float(row[0])]
        readPHI = [float(row[1])]
        readg = [float(row[2])]
        
        
        
        phi_array07.extend(readPHI)
        theta_array07.extend(readTHETA)
        g_array07.extend(readg)

In [3]:
#Initial fit to determine appropriate bounds (prior to imposing convention g1<g2<g3)
#this script uses the gObserved2 function to fit the data.
#it outputs the parameters and the standard deviations.

import scipy as sp
import scipy.optimize

#gObserved(X,gx,gy,gz,psi):
boundsmin = [0,0,0,-PI,-PI,-PI];
boundsmax = [10,10,10,PI,PI,PI];
initial = [1,1,1,0,0,0]
# We first fit with free bounds of g=(0->10)

popt09, pcov09 = sp.optimize.curve_fit(gObserved2, (theta_array09, phi_array09), g_array09,
                                           p0 = initial, bounds = [boundsmin,boundsmax])
popt07, pcov07 = sp.optimize.curve_fit(gObserved2, (theta_array07, phi_array07), g_array07,
                                           p0 = initial, bounds = [boundsmin,boundsmax])
print(popt09)
perr09 = np.sqrt(np.diag(pcov09)) #standard deviation in parameters
print(perr09)

print(popt07)
perr07 = np.sqrt(np.diag(pcov07)) #standard deviation in parameters
print(perr07)

[ 3.92386309  1.43314665  2.25181799  1.78909144 -0.70824475 -1.17435281]
[0.07608128 0.15990252 0.09598678 0.07378393 0.0307165  0.09223703]
[ 1.69182175  1.23936449  0.23575601 -0.36617171  0.2025354  -0.25658597]
[0.0542389  0.05647087 0.1517974  0.04384699 0.30030317 0.24887457]


In [4]:
#this script uses the gObserved2 function to fit the data.
#it outputs the parameters and the standard deviations.

import scipy as sp
import scipy.optimize

#gObserved(X,gx,gy,gz,psi):
boundsmin09 = [0,2,3,-PI,-PI,-PI];
boundsmax09 = [2,3,4,PI,PI,PI];
initial09 = [0.1,2.23,3,0,0,0]
boundsmin07 = [0.0,0.6,1.4,-PI,-PI,-PI];
boundsmax07 = [0.6,1.4,2.0,PI,PI,PI];
initial07 =   [0.1,1.0,1.5,0,0,0]
# We have first fit with free bounds of g=(0->10) to determine the principle g-factors, 
# We now fit again with bounds such that g1<g2<g3 , where bounds are based on results of In[22].

popt09, pcov09 = sp.optimize.curve_fit(gObserved2, (theta_array09, phi_array09), g_array09,
                                           p0 = initial09, bounds = [boundsmin09,boundsmax09])
popt07, pcov07 = sp.optimize.curve_fit(gObserved2, (theta_array07, phi_array07), g_array07,
                                           p0 = initial07, bounds = [boundsmin07,boundsmax07])

#parameter order is: g1, g2, g4, phi_0, Theta_0, vartheta_0 - (as described in Eqn 1 of main text)

print(popt09)
perr09 = np.sqrt(np.diag(pcov09)) #standard deviation in parameters
print(perr09)

print(popt07)
perr07 = np.sqrt(np.diag(pcov07)) #standard deviation in parameters
print(perr07)

[1.43314615 2.25181999 3.92386213 0.73552217 0.32864842 0.14877939]
[0.15990269 0.09598682 0.07608129 0.02303462 0.11362716 0.06003327]
[ 0.23563339  1.23936785  1.69183195  1.21243894 -0.07699316 -0.04010006]
[0.15184935 0.05647094 0.05423798 0.03908238 0.11675574 0.14990144]


In [10]:
#use the fitted pararmeters to output the best fit experimental datasets.
#Run this line to output the data into .csv files.

#define the fitting variable
#Take thesse numbers from the best fit output = popt.
gx09 = popt09[0]
gy09 = popt09[1]
gz09 = popt09[2]
phi09 = popt09[3]
Theta09 = popt09[4]
vartheta09 = popt09[5]

gx07 = popt07[0]
gy07 = popt07[1]
gz07 = popt07[2];
phi07 = popt07[3]
Theta07 = popt07[4]
vartheta07 = popt09[5]

#number of theory points to simulate
numPoints = 36

#Fill the arrays

#Rotation around x-y sample plane
theta_array = []
phi_array = []
for i in range(numPoints):
    phi = 0 #OP angle
    theta = 2*i*PI/numPoints #IP angle
    theta_array.append(theta)
    phi_array.append(phi)

out09 =  gObserved2((theta_array,phi_array),gx09,gy09,gz09,phi09,Theta09,vartheta09)
out07 =  gObserved2((theta_array,phi_array),gx07,gy07,gz07,phi07,Theta07,vartheta07)

#save the array into the text file
#dont forget to name them correctly!!!

output = open('rotXY09.txt','w')
for i in range(numPoints):
    output.write(str(theta_array[i])+','+str(phi_array[i])+','+str(out09[i])+'\n')
        
output.close()

output = open('rotXY07.txt','w')
for i in range(numPoints):
    output.write(str(theta_array[i])+','+str(phi_array[i])+','+str(out07[i])+'\n')
        
output.close()

#Rotation around x-z sample plane
theta_array = []
phi_array = []
out09 = []
out07 = []
for i in range(numPoints):
    phi = 2*i*PI/numPoints #OP angle
    theta = 0 #IP angle
    theta_array.append(theta)
    phi_array.append(phi)

out09 =  gObserved2((theta_array,phi_array),gx09,gy09,gz09,phi09,Theta09,vartheta09)
out07 =  gObserved2((theta_array,phi_array),gx07,gy07,gz07,phi07,Theta07,vartheta07)

#save the array into the text file
#dont forget to name them correctly!!!

output = open('rotXZ09.txt','w')
for i in range(numPoints):
    output.write(str(theta_array[i])+','+str(phi_array[i])+','+str(out09[i])+'\n')
        
output.close()

output = open('rotXZ07.txt','w')
for i in range(numPoints):
    output.write(str(theta_array[i])+','+str(phi_array[i])+','+str(out07[i])+'\n')
        
output.close()

#Rotation around y-z sample plane
theta_array = []
phi_array = []
for i in range(numPoints):
    phi = 2*i*PI/numPoints #OP angle
    theta = PI/2 #IP angle
    theta_array.append(theta)
    phi_array.append(phi)

out09 =  gObserved2((theta_array,phi_array),gx09,gy09,gz09,phi09,Theta09,vartheta09)
out07 =  gObserved2((theta_array,phi_array),gx07,gy07,gz07,phi07,Theta07,vartheta07)

#save the array into the text file
#dont forget to name them correctly!!!

output = open('rotYZ09.txt','w')
for i in range(numPoints):
    output.write(str(theta_array[i])+','+str(phi_array[i])+','+str(out09[i])+'\n')
        
output.close()

output = open('rotYZ07.txt','w')
for i in range(numPoints):
    output.write(str(theta_array[i])+','+str(phi_array[i])+','+str(out07[i])+'\n')
        
output.close()